In [ ]:
import cv2
import mediapipe as mp
import math
# 손가락 이름
finger_names = {8: "Index", 12: "Middle", 16: "Ring", 20: "Pinky"}

def convertImag(x, y, w, h ,orgImage, overImage ):
    # 이미지 경계 검사 및 조정
    ih, iw, _ = orgImage.shape
    x = max(0, min(x, iw))
    y = max(0, min(y, ih))
    w = max(0, min(w, iw - x))
    h = max(0, min(h, ih - y))
    
    overlay_image = cv2.resize(overImage, (w, h))

    #Separate the color and alpha channels of the overlay image
    overlay_img_color = overlay_image[:, :, :3]  # RGB channels
    overlay_img_alpha = overlay_image[:, :, 3]  # Alpha channel

    # Extract the region of interest (ROI) from the original image
    roi = orgImage[y:y+h, x:x+w]

    # Use the alpha channel as a mask to blend the overlay and ROI
    alpha = overlay_img_alpha / 255.0
    inv_alpha = 1.0 - alpha
    
    for c in range(0, 3):
        roi[:, :, c] = (alpha * overlay_img_color[:, :, c] + inv_alpha * roi[:, :, c])
    
    return roi
# 두 랜드마크 사이의 각도를 계산합니다.
def calculate_angle(p1, p2):

    angle = math.atan2(p2[1] - p1[1], p2[0] - p1[0])
    return math.degrees(angle)

# 손가락의 상태를 분석하는 함수
def analyze_finger_pose(landmarks):
 # 각 손가락의 끝과 그 다음 관절 인덱스
    finger_tips = [8, 12, 16, 20]  # 엄지 제외
    lower_joints = [7, 11, 15, 19]
    
    # 손가락 상태
    finger_status = {"Index": "", "Middle": "", "Ring": "", "Pinky": ""}
    #리턴 스트링
    ret_message =''

    # 엄지는 별도 처리 (엄지 끝: 4, 엄지 첫 번째 관절: 3)
    thumb_tip = landmarks[4]
    thumb_joint = landmarks[3]
 
    #finger_status["Thumb"] = "Open" if thumb_tip[1] <= thumb_joint[1] else "Closed"  # x , y , z값이 
    # 엄지 손가락 끝과 첫 번째 관절 사이의 각도 계산
    thumb_angle = calculate_angle(thumb_tip, thumb_joint)
    finger_status["Thumb"] = "Open" if thumb_angle > 90 else "Closed"  # x , y , z값이 
    #finger_status["Thumb"] = thumb_angle 
    for tip, joint in zip(finger_tips, lower_joints):
        # 각 손가락에 대해
        finger_tip = landmarks[tip] # landmarks[0]
        # print(landmarks[8])
        # print(finger_tip)
        lower_joint = landmarks[joint]
        if finger_tip[1] < lower_joint[1]:
            finger_status[finger_names[tip]] = "Open" # fimger_names[8]은 index손가락의 이름을 반환함(즉 finger_name은 랜드마크 인덱스와 손가락 이름 간의 매핑을 가지고 있음 )
        else:
            finger_status[finger_names[tip]] = "Closed"

    if finger_status["Thumb"] == "Open": #엄지척
        ret_message = 'ONE'
    elif finger_status['Index'] == 'Closed' and finger_status['Middle'] == 'Closed' and \
       finger_status['Ring'] == 'Open' and finger_status['Pinky'] == 'Open':
        ret_message = 'SAD'
    elif finger_status['Index'] == 'Open' and finger_status['Middle'] == 'Open' and \
       finger_status['Ring'] == 'Closed' and finger_status['Pinky'] == 'Closed':
        ret_message = 'BRI'
    else:
        ret_message = 'NON'
    return ret_message
#    return finger_status

# Mediapipe 모듈 초기화
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils ## 랜드마크 연결선 그리기 위해 사용 
mp_hands = mp.solutions.hands

face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)
hands = mp_hands.Hands(min_detection_confidence=0.5)

# 슬픈 표정 이미지 로드 (예: 'sad_face.png')
sad_face_img = cv2.imread('sad_face.png', -1)  # -1은 알파 채널 포함
# 웃는 이미지 로드 (예: 'smile.png')
smile_img = cv2.imread('smile.png', -1)  # -1은 알파 채널 포함
# 엄지척 이미지 로드 (예: 'thumb.png')
thumb_img = cv2.imread('thumb.png', -1)  # -1은 알파 채널 포함

hhul_img = cv2.imread('hhul.png', -1)  # -1은 알파 채널 포함

# 웹캠 설정
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # Mediapipe에 사용할 이미지 형식으로 변환
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # 얼굴과 손 감지 수행
    face_results = face_detection.process(image)
    hand_results = hands.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    ih, iw, _ = image.shape

    # 얼굴과 손의 위치를 분석
    try:
        if face_results.detections and hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                # 랜드마크와 연결선 그리기
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2) 
                )  # img: 그림 그릴 대상 이미지/ hand_landmarks:손에 대한 랜드마크 정보/ mp.hand.HAND_CONNECTIONS: 랜드마크 간 연결정보/ 초록색점 빨간색선 
                # 손가락 랜드마크를 리스트로 변환
                landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark] # hand_landmarks.landmark에는 손의 각 랜드마크에 대한 정보가 포함되어 있습
                # print(landmarks) #[(0.1,0.2,0.3), (0.4,0.5,0.6), .....]
                # 손가락 포즈 분석
                finger_pose = analyze_finger_pose(landmarks)
                if finger_pose == 'SAD':
                    overlay_image = sad_face_img
                elif finger_pose == 'ONE':
                    overlay_image = thumb_img
                elif finger_pose == 'BRI':
                    overlay_image = smile_img
                else:
                    overlay_image = hhul_img

                # # 손가락 포즈 분석
                # finger_pose = analyze_finger_pose(landmarks)
                # # 분석 결과를 화면에 표시
                # y = 20
                # for finger, status in finger_pose.items():
                #     cv2.putText(image, f"{finger}: {status}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
                #     y += 20


            for detection in face_results.detections:
                bboxC = detection.location_data.relative_bounding_box
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

                image[y:y+h, x:x+w] = convertImag(x, y, w, h, image, overlay_image)
        
    # 결과 이미지 표시
        cv2.imshow('Hand Over Face Detection', image)

    # ESC를 누르면 종료
        if cv2.waitKey(5) & 0xFF == 27:
            break
    except ValueError as e: 
        print(f"ValueError Occur: {e}")
    except Exception as e:
        print(f"unexpected Error occur:{e}")
    finally:
        pass

cap.release()
cv2.destroyAllWindows()
